# Tutorial Basic

In [1]:
import pandas as pd
import numpy as np

from pyelegantsdds.elegantrun import ElegantRun
from pyelegantsdds.sdds import SDDS

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# path to singularity container with parallel elegant installed
sif = "/home/mti/gitlab-hzb/containers/bin/pelegant.sif"

## Setup example Elegant Lattice

In [4]:
# lattice element definitions
elements ={
    "QF": {"type" : "KQUAD", "L": 0.342, "K1":  0.4900, "N_KICKS": 16},
    "QD": {"type" : "KQUAD", "L": 0.668, "K1": -0.4999, "N_KICKS": 16},
    "D":  {"type" : "DRIF" , "L": 3.5805},
    "W1": {"type" : "WATCH", "filename":"\"%s-%03ld.w1\"","mode": "coordinates"}
}

FODOstr = "! FODO cell.\n\n"
stringlist = ["{:6}: {}".format(k,", ".join(["{}={:15.12f}".format(kk,vv) 
                                             if not isinstance(vv,str)
                                             else "{}={}".format(kk,vv)
                                             if kk!="type" else "{}".format(vv) for kk,vv in v.items()])) 
              for k,v in elements.items()]
line     = ["W1","QF","D","QD","D","QF"]
linestr  = "{:6}: LINE=({})".format("FODO",",".join(line))
FODOstr += "\n".join(stringlist)
FODOstr += "\n\n"
FODOstr += linestr

print(FODOstr)

with open("FODO.lte","w") as f:
    f.write(FODOstr)
    
# set lattice for the rest of the tutorial
lattice = "FODO.lte"

! FODO cell.

QF    : KQUAD, L= 0.342000000000, K1= 0.490000000000, N_KICKS=16.000000000000
QD    : KQUAD, L= 0.668000000000, K1=-0.499900000000, N_KICKS=16.000000000000
D     : DRIF, L= 3.580500000000
W1    : WATCH, filename="%s-%03ld.w1", mode=coordinates

FODO  : LINE=(W1,QF,D,QD,D,QF)


## ElegantRun class

The ElegantRun class is used to setup, control and run the Elegant simulations.

In [5]:
# load Elegant similator
er = ElegantRun(sif,lattice, parallel=True, use_beamline="FODO", energy=1700.00)

The *kwargs* argument is used to extract the settings for the simulations and building of the Elegant command file (temp.ele) - minimum required arguments are:
* use_beamline
* energy

Most command have default argument that can be changed by use of the *kwargs*, in this way simple simulations can be set up quickly and in just a few lines of python code. More complicated examples, using *kwargs* can be found in the **advanced tutorials**.

In [6]:
er.kwargs

{'use_beamline': 'FODO', 'energy': 1700.0}

The *parallel* argument allows to choose between running **serial** or **parallel** Elegant.

In [7]:
er.parallel

True

The latticefile used in the simulations is saved in the *lattice* property.

In [8]:
er.lattice

'FODO.lte'

## Internal elegant_command class

The ElegantRun class uses internally another class that takes care of organizing the Elegant control file. It allows to customize the control file for the simulations.

In [9]:
er.commandfile.commandlist, er.commandfile.history

([], {})

This class contains several commands to build a contral file.

In [10]:
# adds basic run_setup
er.add_basic_setup()

In [11]:
er.commandfile.commandlist, er.commandfile.history

([{'NAME': 'run_setup',
   'NOTE': '',
   'lattice': 'FODO.lte',
   'use_beamline': 'FODO',
   'p_central_mev': 1700.0,
   'default_order': 1,
   'concat_order': 3,
   'rootname': 'temp',
   'parameters': '%s.params',
   'semaphore_file': '%s.done',
   'magnets': '%s.mag'}],
 {})

In [12]:
# the write command writes the commandlist to an .ele file
er.commandfile.write(outputfilename='basic_test.ele')

In [13]:
with open('basic_test.ele','r') as f:
    cmd = f.read()
    
print(cmd)

&run_setup
	lattice             = FODO.lte,
	use_beamline        = FODO,
	p_central_mev       = 1700.0,
	default_order       = 1,
	concat_order        = 3,
	rootname            = temp,
	parameters          = %s.params,
	semaphore_file      = %s.done,
	magnets             = %s.mag,
&end




In [14]:
# modify a command
# mode chooses the command (first, last or index[int])
er.commandfile.modifyCommand('run_setup', mode='last', default_order=2)

No such command run_setup can be found


In [15]:
# notice the change of default_order
er.commandfile.commandlist ,er.commandfile.history

([],
 {0: [{'NAME': 'run_setup',
    'NOTE': '',
    'lattice': 'FODO.lte',
    'use_beamline': 'FODO',
    'p_central_mev': 1700.0,
    'default_order': 1,
    'concat_order': 3,
    'rootname': 'temp',
    'parameters': '%s.params',
    'semaphore_file': '%s.done',
    'magnets': '%s.mag'}],
  1: [{'NAME': 'run_setup',
    'NOTE': '',
    'lattice': 'FODO.lte',
    'use_beamline': 'FODO',
    'p_central_mev': 1700.0,
    'default_order': 1,
    'concat_order': 3,
    'rootname': 'temp',
    'parameters': '%s.params',
    'semaphore_file': '%s.done',
    'magnets': '%s.mag'}]})

In [16]:
# clear the command list - updates history
er.commandfile.clear()

In [17]:
er.commandfile.commandlist ,er.commandfile.history

([],
 {0: [{'NAME': 'run_setup',
    'NOTE': '',
    'lattice': 'FODO.lte',
    'use_beamline': 'FODO',
    'p_central_mev': 1700.0,
    'default_order': 1,
    'concat_order': 3,
    'rootname': 'temp',
    'parameters': '%s.params',
    'semaphore_file': '%s.done',
    'magnets': '%s.mag'}],
  1: [{'NAME': 'run_setup',
    'NOTE': '',
    'lattice': 'FODO.lte',
    'use_beamline': 'FODO',
    'p_central_mev': 1700.0,
    'default_order': 1,
    'concat_order': 3,
    'rootname': 'temp',
    'parameters': '%s.params',
    'semaphore_file': '%s.done',
    'magnets': '%s.mag'}]})

In [18]:
# one can also repeat a command
er.commandfile.addCommand('vary_element',name='*',item='K1', 
                          initial=1.0, final=1.5, index_number=0,index_limit=2)
er.commandfile.repeatCommand('vary_element',mode='last')

In [19]:
er.commandfile.commandlist ,er.commandfile.history

([{'NAME': 'vary_element',
   'NOTE': '',
   'name': '*',
   'item': 'K1',
   'initial': 1.0,
   'final': 1.5,
   'index_number': 0,
   'index_limit': 2},
  {'NAME': 'vary_element',
   'NOTE': '',
   'name': '*',
   'item': 'K1',
   'initial': 1.0,
   'final': 1.5,
   'index_number': 0,
   'index_limit': 2}],
 {0: [{'NAME': 'run_setup',
    'NOTE': '',
    'lattice': 'FODO.lte',
    'use_beamline': 'FODO',
    'p_central_mev': 1700.0,
    'default_order': 1,
    'concat_order': 3,
    'rootname': 'temp',
    'parameters': '%s.params',
    'semaphore_file': '%s.done',
    'magnets': '%s.mag'}],
  1: [{'NAME': 'run_setup',
    'NOTE': '',
    'lattice': 'FODO.lte',
    'use_beamline': 'FODO',
    'p_central_mev': 1700.0,
    'default_order': 1,
    'concat_order': 3,
    'rootname': 'temp',
    'parameters': '%s.params',
    'semaphore_file': '%s.done',
    'magnets': '%s.mag'}]})

In [20]:
# clearing the commandfilelist and history
er.commandfile.clear()
er.commandfile.clearHistory()

In [21]:
er.commandfile.commandlist ,er.commandfile.history

([], {})

## Basic commands

In [22]:
# should always be used first
er.add_basic_setup()
er.commandfile.commandlist ,er.commandfile.history

([{'NAME': 'run_setup',
   'NOTE': '',
   'lattice': 'FODO.lte',
   'use_beamline': 'FODO',
   'p_central_mev': 1700.0,
   'default_order': 1,
   'concat_order': 3,
   'rootname': 'temp',
   'parameters': '%s.params',
   'semaphore_file': '%s.done',
   'magnets': '%s.mag'}],
 {})

In [23]:
er.add_basic_twiss()

In [24]:
er.commandfile.commandlist ,er.commandfile.history

([{'NAME': 'run_setup',
   'NOTE': '',
   'lattice': 'FODO.lte',
   'use_beamline': 'FODO',
   'p_central_mev': 1700.0,
   'default_order': 1,
   'concat_order': 3,
   'rootname': 'temp',
   'parameters': '%s.params',
   'semaphore_file': '%s.done',
   'magnets': '%s.mag'},
  {'NAME': 'twiss_output',
   'NOTE': '',
   'filename': '%s.twi',
   'matched': 1,
   'radiation_integrals': 1}],
 {})

In [25]:
er.add_vary_element()
er.commandfile.commandlist ,er.commandfile.history

([{'NAME': 'run_setup',
   'NOTE': '',
   'lattice': 'FODO.lte',
   'use_beamline': 'FODO',
   'p_central_mev': 1700.0,
   'default_order': 1,
   'concat_order': 3,
   'rootname': 'temp',
   'parameters': '%s.params',
   'semaphore_file': '%s.done',
   'magnets': '%s.mag'},
  {'NAME': 'twiss_output',
   'NOTE': '',
   'filename': '%s.twi',
   'matched': 1,
   'radiation_integrals': 1},
  {'NAME': 'vary_element',
   'NOTE': '',
   'name': '*',
   'item': 'L',
   'intial': 0.0,
   'final': 0.0,
   'index_number': 0,
   'index_limit': 1}],
 {})

In [26]:
er.add_watch_at_start()

In [27]:
er.commandfile.commandlist ,er.commandfile.history

([{'NAME': 'run_setup',
   'NOTE': '',
   'lattice': 'FODO.lte',
   'use_beamline': 'FODO',
   'p_central_mev': 1700.0,
   'default_order': 1,
   'concat_order': 3,
   'rootname': 'temp',
   'parameters': '%s.params',
   'semaphore_file': '%s.done',
   'magnets': '%s.mag'},
  {'NAME': 'twiss_output',
   'NOTE': '',
   'filename': '%s.twi',
   'matched': 1,
   'radiation_integrals': 1},
  {'NAME': 'vary_element',
   'NOTE': '',
   'name': '*',
   'item': 'L',
   'intial': 0.0,
   'final': 0.0,
   'index_number': 0,
   'index_limit': 1},
  {'NAME': 'insert_elements',
   'NOTE': '',
   'name': 'W',
   's_start': -1,
   's_end': -1,
   'skip': 1,
   'insert_before': 0,
   'add_at_end': 0,
   'add_at_start': 1,
   'element_def': '"W: WATCH, FILENAME=\\"%s-%03ld.wq\\", mode=\\"coordinates\\""'}],
 {})

In [28]:
er.add_basic_controls()
er.commandfile.commandlist ,er.commandfile.history

([{'NAME': 'run_setup',
   'NOTE': '',
   'lattice': 'FODO.lte',
   'use_beamline': 'FODO',
   'p_central_mev': 1700.0,
   'default_order': 1,
   'concat_order': 3,
   'rootname': 'temp',
   'parameters': '%s.params',
   'semaphore_file': '%s.done',
   'magnets': '%s.mag'},
  {'NAME': 'twiss_output',
   'NOTE': '',
   'filename': '%s.twi',
   'matched': 1,
   'radiation_integrals': 1},
  {'NAME': 'vary_element',
   'NOTE': '',
   'name': '*',
   'item': 'L',
   'intial': 0.0,
   'final': 0.0,
   'index_number': 0,
   'index_limit': 1},
  {'NAME': 'insert_elements',
   'NOTE': '',
   'name': 'W',
   's_start': -1,
   's_end': -1,
   'skip': 1,
   'insert_before': 0,
   'add_at_end': 0,
   'add_at_start': 1,
   'element_def': '"W: WATCH, FILENAME=\\"%s-%03ld.wq\\", mode=\\"coordinates\\""'},
  {'NAME': 'run_control', 'NOTE': ''},
  {'NAME': 'bunched_beam', 'NOTE': ''},
  {'NAME': 'track', 'NOTE': ''}],
 {})

In [29]:
# as example write to ele file and print 
er.commandfile.write('basic.ele')

with open('basic.ele', 'r') as f:
    cmd = f.read()
    
print(cmd)

&run_setup
	lattice             = FODO.lte,
	use_beamline        = FODO,
	p_central_mev       = 1700.0,
	default_order       = 1,
	concat_order        = 3,
	rootname            = temp,
	parameters          = %s.params,
	semaphore_file      = %s.done,
	magnets             = %s.mag,
&end

&twiss_output
	filename            = %s.twi,
	matched             = 1,
	radiation_integrals = 1,
&end

&vary_element
	name                = *,
	item                = L,
	intial              = 0.0,
	final               = 0.0,
	index_number        = 0,
	index_limit         = 1,
&end

&insert_elements
	name                = W,
	s_start             = -1,
	s_end               = -1,
	skip                = 1,
	insert_before       = 0,
	add_at_end          = 0,
	add_at_start        = 1,
	element_def         = "W: WATCH, FILENAME=\"%s-%03ld.wq\", mode=\"coordinates\"",
&end

&run_control
&end

&bunched_beam
&end

&track
&end




In [30]:
er.clearAll()

In [31]:
er.add_watch(type='KQUAD',insert_before=1, 
             element_def = '"WQ: WATCH, FILENAME=\\"%s-%03ld.wq\\", mode=\\"coordinates\\""')
er.commandfile.commandlist

[{'NAME': 'insert_elements',
  'NOTE': '',
  'type': 'KQUAD',
  's_start': -1,
  's_end': -1,
  'skip': 1,
  'insert_before': 1,
  'add_at_end': 0,
  'add_at_start': 0,
  'element_def': '"WQ: WATCH, FILENAME=\\"%s-%03ld.wq\\", mode=\\"coordinates\\""'}]

In [32]:
er.clearAll()

Some variations of the basic commands exist and will be shown in more detail in the **advanced** tutorial.

In [33]:
er.add_vary_element_from_file()
er.commandfile.commandlist

External filename missing.


[]

In [34]:
er.add_vary_element_from_file(enumeration_file='scan.sdds')
er.commandfile.commandlist

[{'NAME': 'vary_element',
  'NOTE': '',
  'name': '*',
  'item': 'L',
  'index_number': 0,
  'index_limit': 1,
  'enumeration_file': 'scan.sdds',
  'enumeration_column': None}]

In [35]:
er.clearAll()

## Template Commands

### Twiss

In [36]:
# twiss
twidata, twipar = er.findtwiss()

# for example
print("Nux       : {:12.6f}".format(twipar.nux))
print("Nuy       : {:12.6f}".format(twipar.nuy))
print("dNux / dp : {:12.6f}".format(twipar['dnux/dp']))
print("dNuy / dp : {:12.6f}".format(twipar['dnuy/dp']))

Nux       :     0.235726
Nuy       :     0.234332
dNux / dp :    -0.000000
dNuy / dp :    -0.000000


In [37]:
# twiss data is returned as dataframe
twidata

,s,betax,alphax,psix,etax,etaxp,xAperture,betay,alphay,psiy,...,pCentral0,ElementName,ElementOccurence,ElementType,ChamberShape,dI1,dI2,dI3,dI4,dI5
0,0.0000,14.018177,0.000000e+00,0.000000,0.0,0.0,10.0,2.856023,9.761713e-17,0.000000,...,3326.816296,_BEG_,1,MARK,NaN,0.0,0.0,0.0,0.0,0.0
1,0.0000,14.018177,0.000000e+00,0.000000,0.0,0.0,10.0,2.856023,9.761713e-17,0.000000,...,3326.816296,W1,1,WATCH,?,0.0,0.0,0.0,0.0,0.0
2,0.3420,13.238180,2.236957e+00,0.024869,0.0,0.0,10.0,3.064602,-6.214851e-01,0.116974,...,3326.816296,QF,1,KQUAD,?,0.0,0.0,0.0,0.0,0.0
3,3.9225,3.033641,6.130754e-01,0.625301,0.0,0.0,10.0,13.314053,-2.241090e+00,0.712004,...,3326.816296,D,1,DRIF,?,0.0,0.0,0.0,0.0,0.0
4,4.5905,3.033641,-6.130754e-01,0.855812,0.0,0.0,10.0,13.314053,2.241090e+00,0.760351,...,3326.816296,QD,1,KQUAD,?,0.0,0.0,0.0,0.0,0.0
5,8.1710,13.238180,-2.236957e+00,1.456244,0.0,0.0,10.0,3.064602,6.214851e-01,1.355380,...,3326.816296,D,2,DRIF,?,0.0,0.0,0.0,0.0,0.0
6,8.5130,14.018177,1.665335e-16,1.481113,0.0,0.0,10.0,2.856023,2.498002e-16,1.472354,...,3326.816296,QF,2,KQUAD,?,0.0,0.0,0.0,0.0,0.0


In [38]:
# twiss header as series
pd.DataFrame(twipar)

,ParameterValue
ParameterName,
Step,0.000000
nux,0.235726
dnux/dp,-0.000000
dnux/dp2,0.000000
dnux/dp3,0.000000
...,...
Sdelta0,NaN
taudelta,NaN
Jdelta,NaN


### Find  matrices

In [39]:
er = ElegantRun(sif,lattice, parallel=True, use_beamline="FODO", energy=1700.00)
tup = er.find_matrices(SDDS_output_order=4)

AssertionError: 

In [ ]:
tup[0]

In [ ]:
tup[1]

In [ ]:
tup[2]

In [ ]:
tup[3]

In [ ]:
tup[4]

### Tracking

#### Sinlge particle simple

In [ ]:
er = ElegantRun(sif,lattice, parallel=True, use_beamline="FODO", energy=1700.00)
er.simple_single_particle_track(n_passes=100, coord=np.array([1e-5,0,0,0,0]))

In [ ]:
# quick plot
sddsp = SDDS(sif,"temp-001.w1",0)

sddsp.sddsplot_base(
    columnNames="x,xp",
#     file="temp-001.w1",
    graph="symb,vary=subtype,fill",
    device='png',
    output="FODO_single_particle.png"
)

![title](FODO_single_particle.png)

In [ ]:
er.kwargs.get('energy')

In [ ]:
# generate rectangular coordinate input file - auto
# if pcentralmev is not given the energy value in er.kwargs will be used
er.generate_sdds_particle_inputfile(grid_type='rectangular', p_min=1e-6, p_max=1e-2, 
                                    pcentralmev=er.kwargs.get('energy'),
                                    NPOINTS=7
                                   )

In [ ]:
sddsp = SDDS(sif,"temp_particles_input.bin",0)
df = sddsp.readParticleData()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df.x,df.xp,df.y,s=1)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$p_x$')
ax.set_zlabel(r'$y$')
plt.tight_layout()
plt.show()

In [ ]:
# track
er.track_simple(n_passes=2**6)

In [ ]:
# quick plot
sddsp = SDDS(sif,"temp-001.w1",0)

sddsp.sddsplot_base(
    columnNames="x,xp",
    file="temp-001.w1",
#     scale="-1e-4,1e-4,0,0",
    graph="symb,vary=subtype,fill",
    order="spectral",
    split="columnBin=particleID",
    device='png',
    output="FODO_multi_particle_rectangular2.png"
)

![title](FODO_multi_particle_rectangular2.png)

In [ ]:
# generate rectangular coordinate input file - auto
# if pcentralmev is not given the energy value in er.kwargs will be used
er = ElegantRun(sif,lattice, parallel=True, use_beamline="FODO", energy=1700.00)
er.generate_sdds_particle_inputfile(grid_type='spherical',  
                                    dim=6,
                                    rmin=1e-6,
                                    rmax=1e-4,
                                    rsteps=5,
                                    phisteps=10,
                                    half=True, # False gives full spheres
                                    pcentralmev=er.kwargs.get('energy'),
                                   )

In [ ]:
sddsp = SDDS(sif,"temp_particles_input.bin",0)
df = sddsp.readParticleData()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.xaxis.labelpad=15
ax.yaxis.labelpad=15
ax.zaxis.labelpad=15

ax.scatter(df.x,df.xp,df.y,s=1)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$p_x$')
ax.set_zlabel(r'$y$')
plt.tight_layout()
plt.show()

In [ ]:
# track
er.track_simple(n_passes=2)

In [ ]:
sddsp = SDDS(sif,"temp-001.w1",0)
df = sddsp.readParticleData()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df.loc[df.Turn==1].x,df.loc[df.Turn==1].xp,df.loc[df.Turn==1].y,s=1)
ax.scatter(df.loc[df.Turn==2].x,df.loc[df.Turn==2].xp,df.loc[df.Turn==2].y,s=1,c='red')

ax.xaxis.labelpad=15
ax.yaxis.labelpad=15
ax.zaxis.labelpad=15

ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$p_x$')
ax.set_zlabel(r'$y$')

plt.title('Sphere (half) after 1 turn')
plt.show()

# Advanced

In [ ]:
# Lattice parTrack from Elegant examples
latstr = """
l1a: DRIF, L=0.2
l1b: DRIF, L=1.531675
lqb: DRIF, L=0.24
l2:  DRIF, L=0.08
l3:  DRIF, L=1.47
l3a: DRIF, L=0.568095770758662
l3b: DRIF, L=0.551904229241338

! LHK is half the effective length of a kicker
lhk:  DRIF, L=0.175
l34a: DRIF, L=0.811087
l34b: DRIF, L=0.308913

l4: DRIF, L=0.08
l5: DRIF, L=0.325

b1: CSBEND, L=0.8, angle=-0.785398163397, e1=-0.445, e2=-0.445, k2=0.1375, k3=100, hgap=0.0225, fint=0.41, integration_order=4, n_kicks=20, nonlinear=1
b2: CSBEND, L=0.8, angle=-0.785398163397, e1=-0.445, e2=-0.445, k2=0.1375, k3=100, hgap=0.0225, fint=0.41, integration_order=4, n_kicks=20, nonlinear=1

q1: KQUAD, L=0.23, k1=1.786022448154, n_kicks=10
q2: KQUAD, L=0.23, k1=2.295915530046, n_kicks=10
q3: KQUAD, l=0.23, k1=0.0
q4: KQUAD, l=0.23, k1=2.270174600496, n_kicks=10

s1h: SEXT, l=0.1, k2=0.0

! vertical chromaticity sextupole plus vertical steering magnet
sdh: KSEXT, l=0.1, k2=5.95873739969822, n_kicks=4

! horizontal chromaticity sextupole plus horizontal steering magnet
sfh: ksextupole, l=0.1, k2=-1.65546424863732, n_kicks=4

s1: line=(s1h,s1h)
sd: line=(sdh,sdh)
sf: line=(sfh,sfh)

! markers for the septum center and the center of the RF straight
msept: marker
mrf:   marker

! quadrant 1, less half of SF
quadr1: line=(l1a,l1b,s1,l2,q1,lqb,b1,lqb,q2,l3a,lhk,lhk,l3b,sd,l4,q3,lqb,b2,lqb,q4,l5)

! quadrant 2, less half of SF
quadr2: line=(l1a,l1b,s1,l2,q1,lqb,b1,lqb,q2,l3a,lhk,lhk,l3b,sd,l4,q3,lqb,b2,lqb,q4,l5)

! quadrant 3, less half of SF
quadr3: line=(l1a,l1b,s1,l2,q1,lqb,b1,lqb,q2,l3,sd,l4,q3,lqb,b2,lqb,q4,l5)

! quadrant 4, less half of SF
quadr4: line=(l1a,l1b,s1,l2,q1,lqb,b1,lqb,q2,l34a,lhk,lhk,l34b,sd,l4,q3,lqb,b2,lqb,q4,l5)

half: line=(quadr2,sf,-quadr2)

ap: rcol,x_max=0.04,y_max=0.01
par: line=(2*half,ap)
return

""".upper()

with open('parTrack.lte','w') as f:
    f.write(latstr)

In [40]:
lattice = 'parTrack.lte'

In [41]:
er = ElegantRun(sif,lattice, parallel=True, use_beamline="par", energy=880.00)
er.fma(xmin=-0.05,xmax=0.05,ymin=0,ymax=0.02,nx=51,ny=51, n_passes=256)

In [42]:
sddsp = SDDS(sif,'temp.fma',0)

In [43]:
sddsp.sddsplot_fma(
    file="temp.fma",
    device='png',
    output="partrack_fma_diffusionrate.png"
)

TypeError: pyelegantsdds.sdds.SDDSCommand.getCommand() got multiple values for keyword argument 'file'

![title](partrack_fma_diffusionrate.png)

In [ ]:
sddsp.sddsplot_tunediagram(scale="0,1,0,1", device='png',output="partrack_tune.png" )

![title](partrack_tune.png)

In [ ]:
er = ElegantRun(sif,lattice, parallel=True, use_beamline="par", energy=880)
# parallel only supports n-line
er.dynap(xmin=-0.1,xmax=0.1,ymin=0,ymax=0.1,nx=21,ny=21,n_lines=51)

In [ ]:
sddsp = SDDS(sif,"temp.aper",0)
sddsp.process_scan()

In [ ]:
sddsp.readParticleData()

In [ ]:
sddsp.sddsplot_base(col="x,y",file1=sddsp.filename, device='png',output="partrack_aper.png")

![title](partrack_aper.png)

In [ ]:
sddsp.addCommand('sddsplot', col='x,y', file=sddsp.filename,
                 graph="sym,vary=subtype",
                 order='spectral',split='column=sLost',
                )

In [ ]:
er.commandfile.addCommand('run_setup',lattice='parTrack.lte')

In [ ]:
sddsp.getColumnList()

In [ ]:
sddsp.getParameterList()

In [ ]:
sddsp.runCommand()

In [ ]:
sddsp.command_history

In [ ]:
 datasetdc = {
        "Q1" : [1.786022448154-0.01,1.786022448154+0.01],
        "Q2" : [2.295915530046-0.01,2.295915530046+0.01],
     
 }

er = ElegantRun(sif,lattice, parallel=True, use_beamline="par", energy=880.00)
er.generate_sdds_particle_inputfile(grid_type='rectangular', p_min=1e-6, p_max=1e-2, 
                                    pcentralmev=er.kwargs.get('energy'),
                                    NPOINTS=3
                                   )
er.track_vary(varydict=datasetdc,varyitemlist=['k1',"k1"],n_passes=2**4, add_watch_start=True, mode='row')

In [ ]:
sdds = SDDS(sif, "temp-001.wq",0)
sdds.process_scan()

In [ ]:
data = sdds.readParticleData()
data


In [ ]:
sdds = SDDS(sif, "temp.sdds",0)

In [ ]:
sdds.getColumnValues()

In [ ]:
# quick plot
sddsp = SDDS(sif,"temp-001.wq",0)

sddsp.sddsplot_base(
    columnNames="x,xp",
    file="temp-001.w1",
#     scale="-1e-4,1e-4,0,0",
    graph="symb,vary=subtype,fill",
    order="spectral",
    split="columnBin=particleID",
#     device='png',
#     output="FODO_multi_particle_rectangular2.png"
)

In [ ]:
import matplotlib.pyplot as plt

for i in [1.0*n for n in range(1,23)]:
    for name, group in data.loc[data.step==i].groupby("particleID"):
        plt.scatter(group["x"],group["xp"],s=1, label=name)
    #plt.xlim(-1,1)
    plt.show()